In [2]:
import yfinance as yf
import pandas as pd
import ta
import numpy as np
import requests


# Data Extraction: 
Extract S&P 500 healthcare data from wikipedia and financial modeling Prep Api

### Get Ticker data

In [4]:
# 📈 Get ticker data from S&P 500
# url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# def get_tickers(url : str) -> pd.DataFrame:
#     ticker_list = pd.read_html(url)
    
#     df = ticker_list[0]
#     cols = ["Symbol", "GICS Sector"]
    
#     df = df[cols]
#     df = df.rename(columns = {
#         "GICS Sector" : "Sector"
#     })
    
    
#     return df

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
alpha_vantage_api_key = "8MT91WAGYTS1AY12"
etf_symbol = "QQQ"

def get_qqq_holdings(api_key: str, symbol: str) -> pd.DataFrame:

    url = f'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200: 
        json_data = response.json()
        holdings_data = json_data["holdings"]
        df = pd.DataFrame(holdings_data)
    else: 
        print("API request to Alpha Vantage failed.")

    return df

df = get_qqq_holdings(alpha_vantage_api_key, etf_symbol)
holdings_list = df['symbol'].to_list()



In [5]:
print(holdings_list)


['NVDA', 'MSFT', 'AAPL', 'AMZN', 'AVGO', 'META', 'NFLX', 'TSLA', 'GOOGL', 'GOOG', 'COST', 'PLTR', 'TMUS', 'AMD', 'CSCO', 'LIN', 'PEP', 'BKNG', 'TXN', 'INTU', 'QCOM', 'SHOP', 'ISRG', 'AMGN', 'ADBE', 'GILD', 'HON', 'APP', 'MU', 'AMAT', 'LRCX', 'CMCSA', 'ADP', 'ADI', 'PANW', 'MELI', 'KLAC', 'SNPS', 'INTC', 'CRWD', 'VRTX', 'SBUX', 'CEG', 'DASH', 'CDNS', 'MSTR', 'CTAS', 'ORLY', 'PDD', 'MDLZ', 'TRI', 'MAR', 'PYPL', 'CSX', 'ASML', 'REGN', 'MRVL', 'ADSK', 'FTNT', 'AXON', 'AEP', 'MNST', 'NXPI', 'ABNB', 'ROP', 'FAST', 'PCAR', 'IDXX', 'PAYX', 'AZN', 'ROST', 'WDAY', 'CPRT', 'KDP', 'EXC', 'BKR', 'ZS', 'DDOG', 'FANG', 'EA', 'XEL', 'CCEP', 'TTWO', 'CSGP', 'CHTR', 'MCHP', 'VRSK', 'CTSH', 'KHC', 'GEHC', 'ODFL', 'DXCM', 'WBD', 'TEAM', 'TTD', 'LULU', 'ON', 'CDW', 'BIIB', 'GFS', 'ARM', 'n/a']


### Extract historical price data from financial modeling prep

In [6]:
base_url = "https://financialmodelingprep.com/api"
data_type = "historical-price-full"
ticker = "AAPL"
API_KEY = "fpeni6cREKszyIrgBHXcgvXpAMDEhfjH"


url = f"{base_url}/v3/{data_type}/{ticker}?apikey={API_KEY}"


In [14]:
def get_historical_price(url: str) -> pd.DataFrame: 
    response = requests.get(url)
    print(response)
    data = response.json()
    print(data)
    symbol, historical_price = data['symbol'], data['historical']
    
    df_price = pd.DataFrame(historical_price)
    s_symbol = pd.Series(symbol, range(len(historical_price))).rename("Ticker")
    df_symbol = pd.DataFrame(s_symbol)
    
    df_sorted = pd.concat([df_symbol, df_price], axis = 1).sort_values(by = "date")
    df = df_sorted.reset_index(drop = True)
    
    return df

aapl_df = yf.download(
    'AAPL',
    start='2022-01-01',
    end='2023-01-01'
)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [64]:
# 🏥📈 Get the historical price for all the healthcare tickers 

# def get_historical_prices(tickers: list) -> pd.DataFrame: 
    
#     base_url = "https://financialmodelingprep.com/api"
#     data_type = "historical-price-full"
#     ticker = "AAPL"
#     API_KEY = "fpeni6cREKszyIrgBHXcgvXpAMDEhfjH"
    
#     df_agg = pd.DataFrame()
    
#     for i in range(len(tickers)):
#         url = f"{base_url}/v3/{data_type}/{tickers[i]}?apikey={API_KEY}"
#         df_temp = get_historical_price(url)
#         df_agg = pd.concat([df_agg, df_temp], axis = 0)
        
#     return df_agg

def get_historical_prices(tickers): 
    # data = yf.download(tickers, start = '2024-01-01')
    # return data
    df = pd.DataFrame()
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        stock_hst = stock.history(period = "5y")
        symbols_s = pd.Series(ticker, range(len(stock_hst))).rename("Ticker")
        symbols_df = pd.DataFrame(data = symbols_s)
        stock_df = pd.concat([symbols_df, stock_hst], axis = 1)
        print(stock_df)
        df = pd.concat([df, stock_df], axis = 0)
    
    # return df
        


In [65]:
get_historical_prices(holdings_list[:3])

                              0        Open        High         Low  \
0                          NVDA         NaN         NaN         NaN   
1                          NVDA         NaN         NaN         NaN   
2                          NVDA         NaN         NaN         NaN   
3                          NVDA         NaN         NaN         NaN   
4                          NVDA         NaN         NaN         NaN   
...                         ...         ...         ...         ...   
2025-08-26 00:00:00-04:00   NaN  180.059998  182.389999  178.809998   
2025-08-27 00:00:00-04:00   NaN  181.979996  182.490005  179.100006   
2025-08-28 00:00:00-04:00   NaN  180.820007  184.470001  176.410004   
2025-08-29 00:00:00-04:00   NaN  178.110001  178.149994  173.149994   
2025-09-02 00:00:00-04:00   NaN  170.000000  172.380005  167.220001   

                                Close       Volume  Dividends  Stock Splits  
0                                 NaN          NaN        NaN        

In [16]:
# 🗄 Save the dataframe to a csv file: 
df.to_csv("healthcare_stocks.csv", index = False)

# Data Transformation

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

### Import the dataset

In [3]:
# Import the dataframe
df = pd.read_csv("healthcare_stocks.csv")

### Create Target Variable
1. Calculate weekly log returns

In [4]:
# Calculate weekly returns:
def calculate_weekly_returns(df: pd.DataFrame) -> pd.DataFrame:
    df['weekly_return'] = df.groupby('Ticker')['adjClose'].pct_change(periods = -5)
    df = df.dropna(subset = ["weekly_return"])
    
    df["weekly_log_return"] = np.log(1 + df["weekly_return"])
    
    return df

In [5]:
df = calculate_weekly_returns(df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_24254/3520592536.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weekly_log_return"] = np.log(1 + df["weekly_return"])


,Ticker,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,weekly_return,weekly_log_return
0,ABT,2020-06-15,87.52,90.35,87.05,89.55,81.96,5667401,5667401,2.03,2.32000,88.6175,"June 15, 20",0.023200,-0.008708,-0.008746
1,ABT,2020-06-16,90.62,91.76,89.14,89.74,82.13,6144086,6144086,-0.88,-0.97109,90.3150,"June 16, 20",-0.009711,-0.017113,-0.017262
2,ABT,2020-06-17,90.42,91.67,90.10,90.90,83.19,4359470,4359470,0.48,0.53086,90.7725,"June 17, 20",0.005309,0.041176,0.040351


2. 🎯 Create Target Variable

In [6]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    
    median_returns = df.groupby('year_week')['weekly_log_return'].median().reset_index()
    median_returns.columns = ['year_week', 'median_weekly_return']
    
    # Merge median returns back to original dataframe
    if 'median_weekly_return' not in df.columns:
        df = pd.merge(df, median_returns, on='year_week', how='left')
        
    df['target'] = (df['weekly_log_return'] > df['median_weekly_return']).astype(int)
    
    return df

In [7]:
df = create_target_variable(df)
df.head(3)

,Ticker,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,weekly_return,weekly_log_return,year_week,median_weekly_return,target
0,ABT,2020-06-15,87.52,90.35,87.05,89.55,81.96,5667401,5667401,2.03,2.32000,88.6175,"June 15, 20",0.023200,-0.008708,-0.008746,2020-24,0.007935,0
1,ABT,2020-06-16,90.62,91.76,89.14,89.74,82.13,6144086,6144086,-0.88,-0.97109,90.3150,"June 16, 20",-0.009711,-0.017113,-0.017262,2020-24,0.007935,0
2,ABT,2020-06-17,90.42,91.67,90.10,90.90,83.19,4359470,4359470,0.48,0.53086,90.7725,"June 17, 20",0.005309,0.041176,0.040351,2020-24,0.007935,1


In [8]:
# 🗂 Create the panel dataset indexed by date and tickers
# This line of code could definitely be simpler.
def create_panel_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df_sorted = df.sort_values(by = "date")
    if "date" in df.columns and "Ticker" in df.columns: 
        # df = df_sorted.set_index(['date', 'Ticker'])
        df = df_sorted
    return df

In [9]:
# This step could be moved to the pipeline
df = create_panel_dataset(df)
df.head(3)

,Ticker,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,weekly_return,weekly_log_return,year_week,median_weekly_return,target
0,ABT,2020-06-15,87.52,90.35,87.05,89.55,81.96,5667401,5667401,2.03,2.32,88.6175,"June 15, 20",0.0232,-0.008708,-0.008746,2020-24,0.007935,0
36928,INCY,2020-06-15,90.49,93.95,90.47,93.51,93.51,1281635,1281600,3.02,3.34,92.4900,"June 15, 20",0.0334,-0.141559,-0.152637,2020-24,0.007935,0
38180,PODD,2020-06-15,178.38,185.16,175.35,184.80,184.80,872043,871900,6.42,3.60,183.4400,"June 15, 20",0.0360,-0.084922,-0.088746,2020-24,0.007935,0


In [10]:
# 🗑 Remove columns that will not be used for the project
def remove_columns(cols: list, df: pd.DataFrame) -> pd.DataFrame:
    for i in range(len(cols)):
        if cols[i] in df.columns:
            df = df.drop(cols[i], axis = 1)
    return df

In [11]:
cols = ["open", "high", "low", "close", "unadjustedVolume", "change", 
        "changePercent", "label", "changeOverTime", "weekly_return", 
       "year_week", "median_weekly_return"]
df = remove_columns(cols, df)
df.head(3)

,Ticker,date,adjClose,volume,vwap,weekly_log_return,target
0,ABT,2020-06-15,81.96,5667401,88.6175,-0.008746,0
36928,INCY,2020-06-15,93.51,1281635,92.4900,-0.152637,0
38180,PODD,2020-06-15,184.80,872043,183.4400,-0.088746,0


2. Calculate moving averages

In [12]:
#📈 Log Moving Averages

def calc_log_moving_avg(window: int, df: pd.DataFrame) -> pd.DataFrame:
    col_name = f'MA_{str(window)}'
    df[col_name] = df.groupby('Ticker')['adjClose'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).mean()
    )
    
    return df


In [13]:
def calc_log_moving_avgs(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        df = calc_log_moving_avg(window, df)
    return df

In [14]:
windows = [200, 100, 50]
df = calc_log_moving_avgs(windows, df)
df.head(3)

,Ticker,date,adjClose,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50
0,ABT,2020-06-15,81.96,5667401,88.6175,-0.008746,0,81.96,81.96,81.96
36928,INCY,2020-06-15,93.51,1281635,92.4900,-0.152637,0,93.51,93.51,93.51
38180,PODD,2020-06-15,184.80,872043,183.4400,-0.088746,0,184.80,184.80,184.80


### Risk and Volatility

In [15]:
def calc_volatility(col_name: str, window: int, df: pd.DataFrame) -> pd.DataFrame: 
    df[col_name] = df.groupby('Ticker')['weekly_log_return'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).std() * np.sqrt(window)
    )
    
    return df

In [16]:
def calc_volatilties(volatility_dict: dict, df: pd.DataFrame) -> pd.DataFrame: 
    for col_name, window in volatility_dict.items():
        df = calc_volatility(col_name, window, df)
    return df

In [17]:
volatility_dict = {
    'vol_1M': 4,    # 1 month ≈ 4 weeks
    'vol_6M': 26,   # 6 months ≈ 26 weeks
    'vol_12M': 52   # 12 months ≈ 52 weeks
}

df = calc_volatilties(volatility_dict, df)
#df = df.dropna(subset = ['vol_1M', 'vol_6M','vol_12M'])

df.head(3)

,Ticker,date,adjClose,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M
0,ABT,2020-06-15,81.96,5667401,88.6175,-0.008746,0,81.96,81.96,81.96,NaN,NaN,NaN
36928,INCY,2020-06-15,93.51,1281635,92.4900,-0.152637,0,93.51,93.51,93.51,NaN,NaN,NaN
38180,PODD,2020-06-15,184.80,872043,183.4400,-0.088746,0,184.80,184.80,184.80,NaN,NaN,NaN


### Short Term Reversal Factors

In [18]:
def calculate_rsi(window, group): 
    rsi = RSIIndicator(close = group['adjClose'], window = window).rsi()
    return rsi

In [19]:
def calculate_rsis(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        col_name = f'RSI_{str(window)}'
        group = df.groupby('Ticker', group_keys = False)
        df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
        
    return df

In [20]:
windows = [3, 9, 14]
df = calculate_rsis(windows, df)
df.head(3)

,Ticker,date,adjClose,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14
0,ABT,2020-06-15,81.96,5667401,88.6175,-0.008746,0,81.96,81.96,81.96,NaN,NaN,NaN,NaN,NaN,NaN
36928,INCY,2020-06-15,93.51,1281635,92.4900,-0.152637,0,93.51,93.51,93.51,NaN,NaN,NaN,NaN,NaN,NaN
38180,PODD,2020-06-15,184.80,872043,183.4400,-0.088746,0,184.80,184.80,184.80,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def calculate_bb(group, window = 20, window_dev = 2):
    return BollingerBands(close = group['adjClose'], window = window, window_dev = window_dev)

In [22]:
def calculate_bbs(bands: list, df:pd.DataFrame) -> pd.DataFrame:
    group = df.groupby('Ticker', group_keys = False)
    for band in bands:
        if band == "hband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
        elif band == "lband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())
            
    return df

In [23]:
bands = ["hband", "lband"]
df = calculate_bbs(bands, df)
df.head(3)

,Ticker,date,adjClose,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband
0,ABT,2020-06-15,81.96,5667401,88.6175,-0.008746,0,81.96,81.96,81.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36928,INCY,2020-06-15,93.51,1281635,92.4900,-0.152637,0,93.51,93.51,93.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38180,PODD,2020-06-15,184.80,872043,183.4400,-0.088746,0,184.80,184.80,184.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Momentum Factor: 

In [24]:
def calculate_momentum(window_months: int, df: pd.DataFrame, trading_days :int = 21) -> pd.DataFrame:
    window_days = window_months * trading_days
    col_name = f"momentum_{str(window_months)}M"
    group = df.groupby('Ticker')
    
    df[col_name] = group['adjClose'].pct_change(periods = window_days)
    
    return df

In [25]:
def calculate_momentums(windows: list, df:pd.DataFrame) -> pd.DataFrame:
    for window in windows:
        df = calculate_momentum(window, df)
    return df

In [26]:
windows = [12, 6, 1]

df = calculate_momentums(windows, df)
# df.head(3)

### Filter Datasets:

In [40]:
# 🗓Choose Each Wednesday and Thursday as the data transformation
def wed_thurs_selector(df, date_col='date', stock_col='Ticker'):
    """
    More efficient implementation for large datasets.
    """
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([stock_col, date_col])
    
    # Get year-week and day of week
    df['year_week'] = df[date_col].dt.strftime('%Y-%U')
    df['day_of_week'] = df[date_col].dt.dayofweek
    
    # Filter to only Wed/Thu
    wed_thu = df[df['day_of_week'].isin([2, 3])].copy()
    
    # For each stock-week, take first Wednesday or first Thursday
    filtered = wed_thu.groupby([stock_col, 'year_week']).first().reset_index()
    
    return filtered.drop(columns=['day_of_week', 'year_week'])
    

In [47]:
def filter_data(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy()
    df = wed_thurs_selector(df)
    return df.dropna(ignore_index = True)

In [49]:
df = filter_data(df)

### Create Custom transformer
E.g. Remove/Fill in Nan values, log transformation, standardization

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

In [51]:
# Perform Log Transformation: 
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature] = np.log(X['adjClose'] / X[feature])
        return X

### 💽Create training and testing variable

In [52]:
 from sklearn.model_selection import train_test_split

In [53]:
# Create x and y variable
y = df["target"]
X = df.drop(columns = ["target"])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

### 🏭 Build a pipeline to transform features

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [64]:
#build pipeline
log_transform_features = ['MA_200', 'MA_100', 'MA_50', 'hband', 'lband']
# cols_to_drop = ['Ticker', 'date', 'weekly_log_return']

cols = ['Ticker', 'date', 'weekly_log_return']

# Example: Drop columns 'col1' and 'col2'
col_dropper = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', cols)
    ],
    remainder='passthrough'  # keeps all other columns
)

pipeline = Pipeline(steps = [ 
    ('log_transformers', LogTransformer(log_transform_features)),
    ('col_dropper', col_dropper), 
    ('scaler', StandardScaler()), 
    ('classifier', LogisticRegression())
])



In [65]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('log_transformers',
                 LogTransformer(features=['MA_200', 'MA_100', 'MA_50', 'hband',
                                          'lband'])),
                ('col_dropper',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_cols', 'drop',
                                                  ['Ticker', 'date',
                                                   'weekly_log_return'])])),
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression())])

In [66]:
y_pred = pipeline.predict(X_test)
print(y_pred)

[0 0 0 ... 1 0 1]


In [67]:
print(f"Model accuracy: {pipeline.score(X_test, y_test)}")

Model accuracy: 0.5158371040723982
